In [ ]:
import carla
import random
import math
import time
import numpy as np
import cv2
import matplotlib.pyplot as plt
from IPython import display

from lane_detection import LaneDetection
from lateral_control import LateralController
from waypoint_prediction import waypoint_prediction, target_speed_prediction
from longitudinal_control import LongitudinalController

In [ ]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
# world = client.get_world()
world = client.load_world('Town03')


weather = carla.WeatherParameters(
    cloudiness=80.0,
    precipitation=30.0,
    sun_altitude_angle=170.0)

world.set_weather(weather)

blueprint_library = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

vehicle_bp = blueprint_library.find('vehicle.tesla.model3')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

spectator = world.get_spectator()
print(vehicle.get_transform().rotation.pitch)
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-8, y=0, z=20), ), carla.Rotation(pitch=-45.000000, yaw=vehicle.get_transform().rotation.yaw, roll=0.000000))
spectator.set_transform(transform)

In [ ]:
# for i in range(30):
#     vehicle_bp = random.choice(blueprint_library.filter('vehicle.*'))
#     npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

# for v in world.get_actors().filter('vehicle.*'):
#     v.set_autopilot(True)

In [ ]:
def process_image(image, vehicle, spectator, LD_module, LatC_module, LongC_module, fig):
    transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-5.5, y=0, z=2.5), ), carla.Rotation(pitch=8.0, yaw=vehicle.get_transform().rotation.yaw, roll=0.000000))
    spectator.set_transform(transform)
    
    i = np.array(image.raw_data)
    i2 = i.reshape((image.height, image.width, 4))
    i3 = i2[:, :, :3]
    
    bev = LD_module.front2bev(i3)
    lane1, lane2 = LD_module.lane_detection(bev)
    waypoints = waypoint_prediction(lane1, lane2, way_type='center')
    target_speed = target_speed_prediction(waypoints)
    
    plt.ion()
    plt.show()
    LD_module.plot_state_lane(bev, fig, waypoints=waypoints)
    display.clear_output(wait=True)
    
    velocity = vehicle.get_velocity()
    speed = np.hypot(velocity.x, velocity.y)

    steering = LatC_module.stanley(waypoints, speed)
    throttle, brake = LongC_module.control(speed, target_speed)


    control = carla.VehicleControl()

    control.steer = steering
    control.throttle = throttle
    control.brake = brake
    vehicle.apply_control(control)

In [ ]:
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '480')
camera_bp.set_attribute('image_size_y', '270')
camera_bp.set_attribute('fov', '64')
camera_init_trans = carla.Transform(carla.Location(x=-5.5, z=2.5), carla.Rotation(pitch=8.0))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle, attachment_type=carla.AttachmentType.SpringArm)

fig = plt.figure()

LD_module = LaneDetection()
LatC_module = LateralController()
LongC_module = LongitudinalController()

camera.listen(lambda image: process_image(image, vehicle, spectator, LD_module, LatC_module, LongC_module, fig))

In [ ]:
camera.stop()